In [37]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud

In [38]:
df=pd.read_csv("Turkish_IMDB_Dataset.csv")

In [39]:
df=df.head(1000)

df["reviewText_raw"] = df["reviewText"]  # Ham metni saklayalım, ileride kıyaslama için bakalım

In [40]:
df.head()

,reviewText,sentiment,reviewText_raw
0,"Diğer yorumculardan biri, sadece 1 Oz bölümünü...",pozitif,"Diğer yorumculardan biri, sadece 1 Oz bölümünü..."
1,Harika bir küçük prodüksiyon. <br /><br />Çeki...,pozitif,Harika bir küçük prodüksiyon. <br /><br />Çeki...
2,Çok sıcak bir yaz hafta sonunda vakit geçirmek...,pozitif,Çok sıcak bir yaz hafta sonunda vakit geçirmek...
3,"Temel olarak, küçük bir çocuğun (Jake) dolabın...",negatif,"Temel olarak, küçük bir çocuğun (Jake) dolabın..."
4,Petter Mattei'nin Love in the Time of Money fi...,pozitif,Petter Mattei'nin Love in the Time of Money fi...


In [41]:
df["word_count_before"] = df["reviewText"].apply(lambda x: len(str(x).split()))
df["word_count_before"].head()

0    228
1    124
2    119
3    115
4    174
Name: word_count_before, dtype: int64

In [42]:
#Normalizing Case Folding

df["reviewText"]=df["reviewText"].str.lower()

#Punctuations
df["reviewText"]=df["reviewText"].str.replace("[^\w\s]"," ",regex=True)

#Numbers
df["reviewText"]=df["reviewText"].str.replace("\d"," ",regex=True)

df[["reviewText","reviewText_raw"]].head()

,reviewText,reviewText_raw
0,diğer yorumculardan biri sadece oz bölümünü...,"Diğer yorumculardan biri, sadece 1 Oz bölümünü..."
1,harika bir küçük prodüksiyon br br çeki...,Harika bir küçük prodüksiyon. <br /><br />Çeki...
2,çok sıcak bir yaz hafta sonunda vakit geçirmek...,Çok sıcak bir yaz hafta sonunda vakit geçirmek...
3,temel olarak küçük bir çocuğun jake dolabın...,"Temel olarak, küçük bir çocuğun (Jake) dolabın..."
4,petter mattei nin love in the time of money fi...,Petter Mattei'nin Love in the Time of Money fi...


In [43]:
#Stop Words
import nltk

sw=stopwords.words("turkish")
df["reviewText"]=df["reviewText"].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))
df[["reviewText","reviewText_raw"]].head()

,reviewText,reviewText_raw
0,diğer yorumculardan sadece oz bölümünü izledik...,"Diğer yorumculardan biri, sadece 1 Oz bölümünü..."
1,harika bir küçük prodüksiyon br br çekim tekni...,Harika bir küçük prodüksiyon. <br /><br />Çeki...
2,sıcak bir yaz hafta sonunda vakit geçirmek bun...,Çok sıcak bir yaz hafta sonunda vakit geçirmek...
3,temel olarak küçük bir çocuğun jake dolabında ...,"Temel olarak, küçük bir çocuğun (Jake) dolabın..."
4,petter mattei nin love in the time of money fi...,Petter Mattei'nin Love in the Time of Money fi...


In [44]:
#Tokenization
#nltk.download("punkt")
from textblob import TextBlob
df["reviewText"].apply(lambda x: TextBlob(x).words).head()


0    [diğer, yorumculardan, sadece, oz, bölümünü, i...
1    [harika, bir, küçük, prodüksiyon, br, br, çeki...
2    [sıcak, bir, yaz, hafta, sonunda, vakit, geçir...
3    [temel, olarak, küçük, bir, çocuğun, jake, dol...
4    [petter, mattei, nin, love, in, the, time, of,...
Name: reviewText, dtype: object

In [49]:
import pandas as pd
from trnlp import TrnlpWord

# Lemmatizasyon fonksiyonu
def lemmatize(text):
    words = text.split()
    lemmatized_words = []

    for word in words:
        tr_word = TrnlpWord()
        tr_word.setword(word)
        lemma = tr_word.get_base  # Kök formunu al
        lemmatized_words.append(lemma)
    
    return " ".join(lemmatized_words)




# Lemmatizasyon işlemi
df["reviewText"] = df["reviewText"].apply(lemmatize)

# Sonucu göster
print(df[["reviewText_raw", "reviewText"]].head())

                                      reviewText_raw  \
0  Diğer yorumculardan biri, sadece 1 Oz bölümünü...   
1  Harika bir küçük prodüksiyon. <br /><br />Çeki...   
2  Çok sıcak bir yaz hafta sonunda vakit geçirmek...   
3  Temel olarak, küçük bir çocuğun (Jake) dolabın...   
4  Petter Mattei'nin Love in the Time of Money fi...   

                                          reviewText  
0  diğer yor sade böl iz sonra bağ söyle hak ben ...  
1  harika bir küçük prodüksiyon çek teknik müteva...  
2  sıcak bir yaz hafta sonunda vakit geç bun hari...  
3  temel ol küçük bir çocuk dolap bir zombi ol dü...  
4  in tim of film iz görsel açı çarpı bir film ba...  


In [50]:
for i in range(3):  # İlk 3 örneği karşılaştır
    print(f"Orijinal: {df['reviewText_raw'][i]}")
    print(f"İşlenmiş: {df['reviewText'][i]}")
    print("-"*50)

Orijinal: Diğer yorumculardan biri, sadece 1 Oz bölümünü izledikten sonra bağlanacağınızı söyledi. Haklılar, çünkü benim başıma gelen tam olarak bu.<br /><br />Oz'da beni ilk etkileyen şey, daha ilk günden itibaren başlayan vahşeti ve yılmayan şiddet sahneleriydi. İnanın bana, bu korkak veya çekingenler için bir dizi değil. Bu dizi uyuşturucu, seks veya şiddet konusunda hiçbir yumruk atmıyor. Kelimenin klasik kullanımıyla hardcore.<br /><br />Oz olarak adlandırılmasının sebebi, Oswald Maksimum Güvenlikli Eyalet Hapishanesi'ne verilen takma ad olmasıdır. Esas olarak, tüm hücrelerin cam ön cephelere sahip olduğu ve içeriye baktığı, bu nedenle gizliliğin gündemde olmadığı deneysel bir hapishane bölümü olan Emerald City'ye odaklanıyor. Em City birçok kişiye ev sahipliği yapıyor... Aryanlar, Müslümanlar, gangsterler, Latinler, Hristiyanlar, İtalyanlar, İrlandalılar ve daha fazlası... bu yüzden kavgalar, ölümcül bakışlar, şaibeli anlaşmalar ve karanlık anlaşmalar asla uzakta değil.<br /><br 

In [51]:
#Stemming

from nltk.stem import PorterStemmer

ps = PorterStemmer()
df["reviewText"] = df["reviewText"].apply(lambda x: " ".join([ps.stem(word) for word in x.split()]))
df[["reviewText","reviewText_raw"]].head()

,reviewText,reviewText_raw
0,diğer yor sade böl iz sonra bağ söyle hak ben ...,"Diğer yorumculardan biri, sadece 1 Oz bölümünü..."
1,harika bir küçük prodüksiyon çek teknik müteva...,Harika bir küçük prodüksiyon. <br /><br />Çeki...
2,sıcak bir yaz hafta sonunda vakit geç bun hari...,Çok sıcak bir yaz hafta sonunda vakit geçirmek...
3,temel ol küçük bir çocuk dolap bir zombi ol dü...,"Temel olarak, küçük bir çocuğun (Jake) dolabın..."
4,in tim of film iz görsel açı çarpı bir film ba...,Petter Mattei'nin Love in the Time of Money fi...


In [52]:
df["word_count_after"] = df["reviewText"].apply(lambda x: len(str(x).split()))
df["word_count_after"].head()

0    184
1     92
2     90
3     90
4    129
Name: word_count_after, dtype: int64

In [53]:
print("Ön işleme öncesi toplam kelime sayısı:", df["word_count_before"].sum())
print("Ön işleme sonrası toplam kelime sayısı:", df["word_count_after"].sum())

print("\nişlem öncesi ortalama kelime sayısı:",df["word_count_before"].mean())
print("\nişlem sonrası ortalama kelime sayısı:",df["word_count_after"].mean())

print("\nOrtalama kelime sayısı farkı:", df["word_count_before"].mean() - df["word_count_after"].mean())

Ön işleme öncesi toplam kelime sayısı: 170310
Ön işleme sonrası toplam kelime sayısı: 128895

işlem öncesi ortalama kelime sayısı: 170.31

işlem sonrası ortalama kelime sayısı: 128.895

Ortalama kelime sayısı farkı: 41.41499999999999


In [54]:
df[["reviewText", "word_count_before", "word_count_after"]].sample(5)  # Rastgele 5 örnek seç

,reviewText,word_count_before,word_count_after
906,inc oku ön belir hiçbir şekil halk karşı bir a...,133,101
49,orta şaşır derec uysal normal standart göre ha...,80,59
533,hayat sade bir film iz ol yaşa uy durmaksızın ...,91,68
439,film yıl sıkı film ol aday bile film dip bat k...,130,90
18,film hatır sinema iz ilk film gör yer yer kara...,88,63


In [55]:
for i in range(3):  # İlk 3 örneği karşılaştır
    print(f"Orijinal: {df['reviewText_raw'][i]}")
    print(f"İşlenmiş: {df['reviewText'][i]}")
    print("-"*50)

Orijinal: Diğer yorumculardan biri, sadece 1 Oz bölümünü izledikten sonra bağlanacağınızı söyledi. Haklılar, çünkü benim başıma gelen tam olarak bu.<br /><br />Oz'da beni ilk etkileyen şey, daha ilk günden itibaren başlayan vahşeti ve yılmayan şiddet sahneleriydi. İnanın bana, bu korkak veya çekingenler için bir dizi değil. Bu dizi uyuşturucu, seks veya şiddet konusunda hiçbir yumruk atmıyor. Kelimenin klasik kullanımıyla hardcore.<br /><br />Oz olarak adlandırılmasının sebebi, Oswald Maksimum Güvenlikli Eyalet Hapishanesi'ne verilen takma ad olmasıdır. Esas olarak, tüm hücrelerin cam ön cephelere sahip olduğu ve içeriye baktığı, bu nedenle gizliliğin gündemde olmadığı deneysel bir hapishane bölümü olan Emerald City'ye odaklanıyor. Em City birçok kişiye ev sahipliği yapıyor... Aryanlar, Müslümanlar, gangsterler, Latinler, Hristiyanlar, İtalyanlar, İrlandalılar ve daha fazlası... bu yüzden kavgalar, ölümcül bakışlar, şaibeli anlaşmalar ve karanlık anlaşmalar asla uzakta değil.<br /><br 